<a href="https://colab.research.google.com/github/MJMortensonWarwick/AnalyticsInPractice2425/blob/main/3_1_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering
In this notebook we will look at some further feature engineering tasks and steps.

We will begin by looking at dealing with missing values

## Imputting Missing Values
We start by creating some data:

In [1]:
import pandas as pd

data = {
    123: {'name': 'Siamak Naderi', 'salary': 34567, 'title': 'AP', 'score': 7, 'sport': 'snooker'},
    345: {'name': 'Vinh Doan', 'salary': 45678, 'title': 'Reader', 'score': 5},
    456: {'name': 'Frances O Brien', 'salary': 56789, 'title': 'Reader', 'score': 6},
    567: {'name': 'Ram Gopal', 'salary': 678910, 'title': 'Prof', 'score': 8},
    789: {'name': 'Michael Mortenson'}
}

df = pd.DataFrame.from_dict(data, orient='index')
# orient index means the index (IDs) will be the rows
df

,name,salary,title,score,sport
123,Siamak Naderi,34567.0,AP,7.0,snooker
345,Vinh Doan,45678.0,Reader,5.0,NaN
456,Frances O Brien,56789.0,Reader,6.0,NaN
567,Ram Gopal,678910.0,Prof,8.0,NaN
789,Michael Mortenson,NaN,NaN,NaN,NaN


Unfortunately I was late submitting my data so most of my records are 'NaN' (i.e. _NULL_). However, obviously I will be critical for any data analysis so we need to populate my data somehow.

The first field/column is 'salary'. We could take the mean average of the column - however, we may note (as is common in salaries) there is a very large value at the end that may skew the data. If we take the mean it will be very influenced by this single value:

In [2]:
# Take the mean value of salary from the dataframe
mean_salary = df['salary'].mean()
print(f"Mean salary is: {mean_salary}")

Mean salary is: 203986.0


As we can see, the mean is more than 3x bigger than the 2nd biggest value, and not particularly representative. A better solution would be to fill the missing value with the median:

In [3]:
# Fill the missing value in salary in the df with median
median_salary = df['salary'].median()
df['salary'] = df['salary'].fillna(median_salary)

df

,name,salary,title,score,sport
123,Siamak Naderi,34567.0,AP,7.0,snooker
345,Vinh Doan,45678.0,Reader,5.0,NaN
456,Frances O Brien,56789.0,Reader,6.0,NaN
567,Ram Gopal,678910.0,Prof,8.0,NaN
789,Michael Mortenson,51233.5,NaN,NaN,NaN


Now we have a much more appropriate value!

How about our 'title' column? Obviously this a text/string value not a numerical value (although we could convert to _[1,2,3]_ as below) so the mean or median won't work. Instead we could use the mode. However, let's first check what values are in there:

In [4]:
# Show all the unique values in df['title']
print(df['title'].unique())

['AP' 'Reader' 'Prof' nan]


As we know, we have three titles and a missing value. But which is the mode (most common)?

In [5]:
# Fill the missing value in title in the df with mode
mode_title = df['title'].mode()[0]
# [0] means return the first in case of multiple records being the most frequent
df['title'] = df['title'].fillna(mode_title)

df

,name,salary,title,score,sport
123,Siamak Naderi,34567.0,AP,7.0,snooker
345,Vinh Doan,45678.0,Reader,5.0,NaN
456,Frances O Brien,56789.0,Reader,6.0,NaN
567,Ram Gopal,678910.0,Prof,8.0,NaN
789,Michael Mortenson,51233.5,Reader,NaN,NaN


Awesome! Next we have 'score'. Unlike 'title' these are numeric, and unlike 'salary' there are no extreme values. We can just use the mean here:

In [6]:
# Fill the missing value in score in the df with mean
mean_score = df['score'].mean()
df['score'] = df['score'].fillna(mean_score)

df

,name,salary,title,score,sport
123,Siamak Naderi,34567.0,AP,7.0,snooker
345,Vinh Doan,45678.0,Reader,5.0,NaN
456,Frances O Brien,56789.0,Reader,6.0,NaN
567,Ram Gopal,678910.0,Prof,8.0,NaN
789,Michael Mortenson,51233.5,Reader,6.5,NaN


Again, all seems very appropriate. What about 'sport'?

Here we have only one value and 4x 'NaN' - this is a lot of filling for little real values. The sensible choice would be to either find a way to get more data here (SMEs) or to just get rid of it. We'll go with option two because we don't like taking to people:

In [7]:
# drop the sport column
df = df.drop('sport', axis=1) # "axis = 1" means columns
df

,name,salary,title,score
123,Siamak Naderi,34567.0,AP,7.0
345,Vinh Doan,45678.0,Reader,5.0
456,Frances O Brien,56789.0,Reader,6.0
567,Ram Gopal,678910.0,Prof,8.0
789,Michael Mortenson,51233.5,Reader,6.5


Great work - a complete dataframe! However, at this point our concious might get the better of us. As important as it is to include 'Michael Mortenson' from a philisophical stand point, essentially all the data we have for him is made-up. Let's do what we should have done from the start and just drop the row:

In [8]:
# drop 'Michael Mortenson' from the dataframe

df = df.drop(789, axis=0) # "axis = 0" means rows
df

,name,salary,title,score
123,Siamak Naderi,34567.0,AP,7.0
345,Vinh Doan,45678.0,Reader,5.0
456,Frances O Brien,56789.0,Reader,6.0
567,Ram Gopal,678910.0,Prof,8.0


## Dealing with Duplicates
Duplicated records can have negative impacts such as biasing our summary statistics or even our models. Generally we just want to remove these:

In [9]:
import pandas as pd

data = {
    123: {'name': 'Ram Gopal', 'salary': 678910, 'title': 'Prof', 'score': 8},
    345: {'name': 'Vinh Doan', 'salary': 45678, 'title': 'Reader', 'score': 5},
    456: {'name': 'Frances O Brien', 'salary': 56789, 'title': 'Reader', 'score': 6},
    567: {'name': 'Ram Gopal', 'salary': 678910, 'title': 'Prof', 'score': 8},
    789: {'name': 'Siamak Naderi', 'salary': 34567, 'title': 'AP', 'score': 7, 'sport': 'snooker'}
}

df = pd.DataFrame.from_dict(data, orient='index')
# orient index means the index (IDs) will be the rows
df

,name,salary,title,score,sport
123,Ram Gopal,678910,Prof,8,NaN
345,Vinh Doan,45678,Reader,5,NaN
456,Frances O Brien,56789,Reader,6,NaN
567,Ram Gopal,678910,Prof,8,NaN
789,Siamak Naderi,34567,AP,7,snooker


To demonstrate the duplicate let's first sort our dataset:

In [10]:
# sort by score and title
# inplace=True will replace the original dataset
df.sort_values(by=['score', 'title'], inplace=True)
df

,name,salary,title,score,sport
345,Vinh Doan,45678,Reader,5,NaN
456,Frances O Brien,56789,Reader,6,NaN
789,Siamak Naderi,34567,AP,7,snooker
123,Ram Gopal,678910,Prof,8,NaN
567,Ram Gopal,678910,Prof,8,NaN


Two Rams. Let's remove one:

In [11]:
df = df[~df.duplicated(keep='first')] # keep the first instance only
df

,name,salary,title,score,sport
345,Vinh Doan,45678,Reader,5,NaN
456,Frances O Brien,56789,Reader,6,NaN
789,Siamak Naderi,34567,AP,7,snooker
123,Ram Gopal,678910,Prof,8,NaN


## Dealing with Categorical Data
Essentially all ML requires our data to be numerical (at the point of analysis) but quite often we get data which is text. While we could just ignore this, it is often desireable to apply some form of transformation to numerical values.

If our data is binary (one of two options) this is quite easy. E.g. let's consider a feature which is "hasPhD?" with a "yes" for people who have one and "no" for the rest. We can easily transform this with an if statement like below (using np.where() for inline calculation of a whole DataFrame but it works the same as an if):

In [12]:
import numpy as np

# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "hasPhD?": ["yes", "yes", "no", "no", "no"]})
print(df)

print("\n")

# change "hasPhD? to numerical values"
df["hasPhD?"] = np.where(df["hasPhD?"] == "yes", 1, 0)
print(df)

   a hasPhD?
0  1     yes
1  2     yes
2  3      no
3  4      no
4  5      no


   a  hasPhD?
0  1        1
1  2        1
2  3        0
3  4        0
4  5        0


This deals with the binary case - how about cases where there are multiple categories?

In [13]:
# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "businessType": ["Hospitality", "IT", "Education", "IT", "Oil"]})
df

,a,businessType
0,1,Hospitality
1,2,IT
2,3,Education
3,4,IT
4,5,Oil


Clearly we can't apply the same trick. Of course, we could assign 1 to "Hospitatlity", 2 to "IT", 3 to "Education" (etc.) ... but is it really meaningful? Could we say Hospitality + IT = Education? These caclulations show such a scheme is not sensible. Instead we use dummy variables (one hot encoding):

In [14]:
# generate binary values using get_dummies
# 'prefix' is used at the start of the column names
dummy_df = pd.get_dummies(df, columns=["businessType"], prefix=["type_"], dtype='int')
dummy_df

,a,type__Education,type__Hospitality,type__IT,type__Oil
0,1,0,1,0,0
1,2,0,0,1,0
2,3,1,0,0,0
3,4,0,0,1,0
4,5,0,0,0,1


We have created 4x new columns - one for each option - and then a binary value of 1 (if it is in this category) and 0 (for the categories it is not in). However, we may notice that actually we don't need all four columns.

If the category is 'type_Education', then actually we can denote this by just the other three columns being 0. This is a unique value already without the need for the 'type_Education' column showing 1. Let's try the code again, but this time with just three columns:

In [15]:
# generate binary values using get_dummies
# 'prefix' is used at the start of the column names
# drop first removes the first category type as it is redundant
dummy_df = pd.get_dummies(df, columns=["businessType"], prefix=["type_"], dtype='int', drop_first=True)
dummy_df

,a,type__Hospitality,type__IT,type__Oil
0,1,1,0,0
1,2,0,1,0
2,3,0,0,0
3,4,0,1,0
4,5,0,0,1


## Splitting Features
Another common scenario is where we have a text field (typically) in one column and want to split it into two. Take this example:

In [16]:
# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "degree": ["MSc BA", "BA Art", "MSc CSM", "BSc CS", "MSc eBM"]})
df

,a,degree
0,1,MSc BA
1,2,BA Art
2,3,MSc CSM
3,4,BSc CS
4,5,MSc eBM


We want to split this data into degree type and subject. There are lots of ways of doing this but we can do it using a deliminator:

In [17]:
# split based on a space (" ") and 'expand' to two columns
df[['degree', 'subject']] = df['degree'].str.split(' ', expand=True)
df

,a,degree,subject
0,1,MSc,BA
1,2,BA,Art
2,3,MSc,CSM
3,4,BSc,CS
4,5,MSc,eBM


## Normalising Data
When using data as features in machine learning we often want to ensure they are on the same scale or otherwise the features that are on the largest scale will have the most influence on the model (e.g. "a" is on a scale of 0 to 1; "b" is on a scale -100 to 10,000 ... "b" will be more influential). There are a few methods to do this:

In [18]:
df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [10000, 100, 20000, 0, 1234567]})
print(df)
print("\n")

# mean normalisation / standardisation
df["salary"] = (df["salary"] - df["salary"].mean()) / df["salary"].std()
print("Mean normalised / standardised")
print(df)
print("\n")

# min-max normalisation
df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [100000, 100, 20000, 0, 123456]})
df["salary"] = (df["salary"] - df["salary"].min()) / (df["salary"].max() -df["salary"].min())
print("Min/max normalised")
print(df)
print("\n")

# robust scaling (via scikit-learn)
# Similar to min-max but uses interquartile-range so is robust to outliers
from sklearn.preprocessing import RobustScaler

df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [100000, 100, 20000, 0, 123456]})

scaler = RobustScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=['a', 'salary'])
print("Robust scaler normalised")
print(df)

   a   salary
0  1    10000
1  2      100
2  3    20000
3  4        0
4  5  1234567


Mean normalised / standardised
   a    salary
0  1 -0.442653
1  2 -0.460692
2  3 -0.424432
3  4 -0.460874
4  5  1.788651


Min/max normalised
   a    salary
0  1  0.810005
1  2  0.000810
2  3  0.162001
3  4  0.000000
4  5  1.000000


Robust scaler normalised
     a    salary
0 -1.0  0.800801
1 -0.5 -0.199199
2  0.0  0.000000
3  0.5 -0.200200
4  1.0  1.035596


Which is best? There's not really a good answer to this and it probably doesn't matter _that_ much. Robust scaler won't be skewed by extreme values so if you see this in your data it makes sense. Normally I would use this or simple min/max normalisation.